In [1]:
## import libraries
import numpy as np
import pandas as pd

In [2]:
## read data
data = pd.read_csv("combined_transactions.csv")

In [3]:
data.head()

,user_id,transaction_type,merchant_type_code,amount_cents,datetime
0,31373,PurchaseActivity,5310,107700,2023-04-15T12:04:02.61636
1,5453,PurchaseActivity,7298,60987,2023-03-14T11:16:29.036731
2,75853,PurchaseActivity,5200,2997,2023-04-15T12:42:08.87193
3,91293,PurchaseActivity,4829,932,2023-05-04T09:25:15.061315
4,91293,PurchaseActivity,4829,943,2023-05-04T09:26:26.80834


In [5]:
## filter and keep only purchase
df = data[data['transaction_type'] == 'PurchaseActivity'].reset_index(drop=True)

In [15]:
## get merchant type code is 5732
target_df = df[df['merchant_type_code'] == 5732].reset_index(drop=True)
target_df['datetime'] = pd.to_datetime(target_df['datetime']).dt.date

In [29]:
## group by date and get the average of amount cents
final_df = target_df.groupby('datetime')['amount_cents'].mean().reset_index()
final_df['datetime'] = pd.to_datetime(final_df['datetime'])
final_df = final_df.set_index('datetime')

In [50]:
## shift the amount cents by 10 days and set it as target value we want to predict
final_df = final_df.resample('D', convention='end').asfreq().fillna(method='ffill').head(30)
final_df['target'] = final_df['amount_cents'].shift(-10).fillna(method='ffill')


In [41]:
final_df.head()

,amount_cents,target
datetime,,
2023-03-14,8330.333333,7080.333333
2023-03-15,65022.833333,67746.000000
2023-03-16,47339.764706,67746.000000
2023-03-17,53997.000000,67746.000000
2023-03-18,53997.000000,7745.250000


In [55]:
from sklearn.linear_model import Ridge

## use ridge model to predict
clf = Ridge(alpha=10)
clf.fit(final_df['amount_cents'].values.reshape(-1, 1), final_df['target'].values)
final_df['prediction'] = clf.predict(final_df['amount_cents'].values.reshape(-1, 1))

In [56]:
final_df.head(20)

,amount_cents,target,prediction
datetime,,,
2023-03-14,8330.333333,7080.333333,47884.545583
2023-03-15,65022.833333,67746.000000,48904.891736
2023-03-16,47339.764706,67746.000000,48586.633570
2023-03-17,53997.000000,67746.000000,48706.449850
2023-03-18,53997.000000,7745.250000,48706.449850
2023-03-19,53997.000000,7930.500000,48706.449850
2023-03-20,53997.000000,17449.500000,48706.449850
2023-03-21,70000.000000,499975.000000,48994.470302
2023-03-22,70000.000000,499975.000000,48994.470302
